In [1]:
import os 
import sys
import inspect

currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir)
sys.path.insert(0, parentdir) 

import pyLLE
import numpy as np
from scipy import constants as cts
import pandas as pd
import pickle as pkl
import plotly.graph_objs as go
import plotly.graph_objs
import plotly.io as pio
import time
pio.renderers.default = "notebook"
pio.templates.default = "seaborn"

************************************************************
Julia solver:
/Users/yakub/Documents/GitHub/pyLLE/pyLLE/ComputeLLE.jl
************************************************************


Comparing the obtained Conversion efficiency from our implementation of bidirectional LLE with the results in Black, Papp: https://opg.optica.org/optica/fulltext.cfm?uri=optica-9-10-1183&id=510242

This will tell us whether the simulations accurately capture the dynamics of a real PhCr system or not

In [2]:
#loop reflector parameters
r=0.0
phi=np.pi
R=np.sqrt(r)*np.exp(1j*phi)
res = dict(
        R=22.5e-6, 
        Qi=2e6, 
        Qc=2e6, 
        γ=1.8199571642510575, 
        dispfile="./Tantala_test_PhCr_Disp.csv", #"./RW1000_H430.csv",
        Bragg=4.2e9 ,#In terms of frequency shift that the pump mode observes due to bragg
        loop_ref_real=np.real(R),
        loop_ref_imag=np.imag(R) #Reflection coefficient induced effectively by the isolator+loop

)

sim = dict(
    Pin=[42e-3], 
    f_pmp=[193.4144890323e12], #pump frequency 1550nm
    φ_pmp=[0], 
    δω=[None], 
    Tscan=0.7e6,
    μ_sim=[-50, 50],
    δω_init= 1e9 * 2 * np.pi,
    δω_end= -6.5e9 * 2 * np.pi,
    num_probe = 5000, 
)


In [3]:
solver = pyLLE.LLEsolver(sim=sim, res=res,debug=True)

In [4]:
fig = solver.Analyze(plot=True)
fig.update_yaxes(range = [-50, 50], title = 'D<sub>int</sub> (GHz)')
fig.show()

In [5]:
solver.disp

+--------------------------------------------------------------------------------------------------------------------+
|  Parameter      Description                       Values                                             Units         |
+--------------------------------------------------------------------------------------------------------------------+
|  ng             [2.113 ... 2.129]                 Group Index                                                      |
|  D              [-8.146 ... -77.926]              Dispersion                                         (ps/nm/km)    |
|  neff           [-2.228 ... 0.718]                Effctive Index                                                   |
|  freq           [94.229 ... 292.229] (THz)        Mode frequency                                     Hz            |
|  freq_sim       [144.414 ... 242.414] (THz)       Mode frequency to match the sim domain             Hz            |
|  Dint           [array for each pump]         

In [6]:
solver.sim

+---------------------------------------------------------------------------------------------------------+
|  Parameter          Description         Values                                      Units               |
+---------------------------------------------------------------------------------------------------------+
|  Pin                [42.000] (mW)       Pump power                                  W                   |
|  Tscan              700000.0            Simulation time length                      unit of round trip  |
|  f_pmp              [193.414] (THz)     Pump frequencies                            Hz                  |
|  φ_pmp              [0]                 Pump phase                                  rad                 |
|  f_center           193.414 (THz)       Center of the sim domain                    Hz                  |
|  δω_init            1.000 (GHz)         Start of the frequency sweep                x2π Hz              |
|  δω_end             -6.500

In [7]:
solver.res

+---------------------------------------------------------------------------------------------------+
|  Parameter          Description                      Values                            Units      |
+---------------------------------------------------------------------------------------------------+
|  R                  22.5                             Ring radius                       µm         |
|  Qi                 2.000 x10^6                      Intrinsic quality factor                     |
|  Qc                 2.000 x10^6                      Coupling quality factor                      |
|  γ                  1.8199571642510575               Non-linear coefficient            W^-1 m^-1  |
|  dispfile           ./Tantala_test_PhCr_Disp.csv     mode/resonance frequency file                |
|  Bragg              2000.0                           Bragg Shift (MHz)                            |
|  Loop Reflector     0j                               reflection constant        

In [8]:
print(f"R = {solver.res.R}")
print(f"Pin = {solver.sim.Pin}")
print(f"D1 = {solver.disp.D1*1e-9/(2*np.pi)} x 2π GHz")

R = 2.25e-05
Pin = [0.042]
D1 = 1000.0000000000006 x 2π GHz


In [9]:
solver.Setup(verbose = True)
solver.SolveTemporal(bin = "/usr/local/bin/julia",verbose=False)
time.sleep(1)
solver.RetrieveData()
solver.sol

-- Solving standard LLE --
	Simulation Parameters
		R = 22.50 µm
		Qi = 2.00 M
		Qc = 2.00 M
		γ = 1.82 
		Bragg Shift = 2000.00 MHz
		loop_ref_real = -0.00 
		loop_ref_imag = 0.00 
	Simulation Parameters
		Pin[0] = 42.00 mW
		f_pmp[0] = 193.41 THz
		Tscan = 0.70 x1e6 Round Trip
		μ_sim = [-50.00,50.00] 
		δω_init = 1.00 x2π GHz
		δω_end = -6.50 x2π GHz
		μ_fit = [None,None] 
		δω_stop = -6.50 x2π GHz
		ind_pump_sweep[0] = 0.00 

HDF5 parameter file can be foud in: /var/folders/k2/n79h9ytx5rq114j_k4jrxv5w0000gn/T/tmp7lwfhl26ParamLLEJulia.h5
----------------------------------------------------------------------
2024-01-23 21:37:42
Computing LLE [**************************************************] 100%

Simulation Time 00h:01min:09.0s
----------------------------------------------------------------------
wtf


+-------------------------------------------------------------------------------------------------+
|  Parameter     Description                          Values                               Units  |
+-------------------------------------------------------------------------------------------------+
|  δfreq         [1.000 ... -6.499] (GHz)             Pump detuning                        Hz     |
|  time          [-0.093 ... 0.607] (μs)              Simualtion time                      s      |
|  Awg           [101 fast time x 5000 slow time]     E. field in time domain in wg,       V/m    |
|  Acav          [101 fast time x 5000 slow time]     E. field in time domain in cav.      V/m    |
|  Ewg           [101 spectra x 5000 slow time]       E. field in freq. domain in wg.      V/m    |
|  Ecav          [101 spectra x 5000 slow time]       E. field in freq. domain in cav.     V/m    |
|  Pcomb         [0.000 ... 0.000] (mW)               Intra-cavity comb power              W      |


In [10]:
fig = go.Figure()

tr = go.Scatter(y=solver.sol.Pcomb * 1e3)

fig.add_trace(tr)
fig.update_layout(
    xaxis_title="LLE sub-sampled step", yaxis_title="Intra-cavity Power (mW)"
)
#fig.add_annotation(x=4500, y=4.9, ax=0, ay=-50, text="Single Soliton Step")
# fig.add_annotation(x=3151, y=7.6, ax=0, ay=-50, text="Two Solitons Step")
#fig.add_annotation(x=1931, y=16, ax=0, ay=80, text="Modulation Instability")
#fig.add_annotation(x=500, y=6, ax=-80, ay=-80, text="Primary Comb")
#fig.show()

In [21]:
ind=3000
solver.PlotCombSpectra(ind).show()